In [19]:
import numpy as np
from numpy.linalg import norm, solve, multi_dot, inv
from scipy.optimize import minimize_scalar

### Exact line search steepest descent method

In [20]:
def exact_steepest_descent(x0):             # input is the starting point
    x = x0                                  # select the starting point
    p = -grad_objective_func(x)             # find descent direction
    i = 1                                   # starting counter for iteration
    while norm(p) > 1e-9:                   # if the norm is not small
        def subproblem1d(alpha):            # define a 1d subproblem 
            return objective_func(x + alpha * p)  
                                            # use minimize_scalar function
        res = minimize_scalar(subproblem1d) # to solve the subproblem
        x = x + res.x * p                   # locate the next iterate
        p = -grad_objective_func(x)         # find next descent direction
        i += 1
    return i,x

### Fixed step length steepest descent method with α = 10−3

In [21]:
def inexact_steepest_descent(x0):           # input is the starting point
    x = x0                                  # select the starting point
    p = -grad_objective_func(x)             # find descent direction
    i = 1                                   # starting counter for iteration
    while norm(p) > 1e-9:                   # if the norm is not small
        x = x + 1e-3 * p                    # locate the next iterate
        p = -grad_objective_func(x)         # find next descent direction
        i += 1
    return i, x

### Newton’s method with fixed step length as 1

In [60]:
def newton_method(x0):                      # input is the starting point
    x = x0                                  # select the starting point
    p = -grad_objective_func(x)             # find descent direction
    h = hessian_func(x)                     # find hessian matrix
    i = 0
    while norm(p) > 1e-9:                   # if the norm is not small
        newton_dir = np.linalg.solve(h, p)  # Newton direction
        x = x + newton_dir                  # locate the next iterate
        p = -grad_objective_func(x)         # find next descent direction
        h = hessian_func(x)                 # find next hessian matrix
        i += 1
    return i, x

### Backtracking steepest descent method with c1 = 10−4, ρ = 0.1

In [23]:
def backtracking(objFunc, gradObjFunc, x0, pk, alpha0, c1, rho):
    currentFuncValue = objFunc(x0)
    currentGradient  = gradObjFunc(x0) 
    alpha            = alpha0 
    # make sure it is a descent direction. 
    assert(np.dot(pk, currentGradient) < 0)
    
    while objFunc(x0 + alpha * pk) - currentFuncValue > c1 * alpha * np.dot(pk, currentGradient):
        alpha = alpha * rho 
    
    return alpha

In [24]:
tol     = 1e-9 
maxIter = 1e6
def backtracking_steepest_decent_method(objFunc, gradObjFunc, x0, tol, maxIter):
    path      = [x0]
    k         = 0
    xk        = x0
    pk        = -gradObjFunc(x0)
    while norm(pk) > tol and k <= maxIter:
        # at xk, find suitable alpha
        alpha = backtracking(objFunc, gradObjFunc, xk, pk, 1, 1e-4 , 0.1)
        xk  = xk + alpha * pk 
        pk  = -gradObjFunc(xk)
        k   = k + 1
        path.append(xk)

    path = np.array(path) # convert to array     
    return xk, k, path

### Backtracking Newton’s method with c1 = 10−4, ρ = 0.9

In [25]:
def backtracking_newton_method(objFunc, gradObjFunc, hessianFunc, x0, tol, maxIter):                      
    path      = [x0]
    k         = 0
    xk        = x0    
    pk        = -np.linalg.solve(hessianFunc(xk), gradObjFunc(xk))        
    while norm(gradObjFunc(xk)) > tol and k <= maxIter:                  
        alpha = backtracking(objFunc, gradObjFunc, xk, pk, 1, 1e-4, 0.9)      
        xk  = xk + alpha * pk 
        pk  = -np.linalg.solve(hessianFunc(xk)  , gradObjFunc(xk))
        k   = k + 1
        path.append(xk)
        
    path = np.array(path) # convert to array        
    return xk, k, path

### Heavyball method with α = 10−3 and β = 0.95

In [26]:
alpha = 1e-3 # fixed step length line search.
beta  = 0.9  # large beta means the ball is very "heavy".
def heavyBall(objFunc, gradObjFunc, x0, tol, maxIter):
    path      = [x0]
    k         = 0
    xk        = x0    
    pk        = -gradObjFunc(xk)
    # Compute the first step separately. 
    if norm(pk) < tol:
        return xk, 0, path
    else:
        k = k + 1
        xk = xk + alpha * pk 
        path.append(xk)
    
    # The rest of iterations
    pk        = -gradObjFunc(xk)
    
    while norm(pk) > tol and k <= maxIter: 
        # use path[-2] since path[-1] is the xk
        xk  = xk + alpha * pk + beta * (xk - path[-2])
        pk  = -gradObjFunc(xk)
        k   = k + 1
        path.append(xk)
        
    path = np.array(path) # convert to array        
    return xk, k, path          

### Trust region method with Cauchy point

In [27]:
def model(x, p):
    return objective_func(x) + np.dot(p, grad_objective_func(x)) + 0.5 * multi_dot([p.T, hessian_func(x), p])


def rho(x, p):
    return (objective_func(x) - objective_func(x + p)) / (model(x, np.array([0, 0]))  - model(x, p))

# framework for trust region method, we use the Hessian as the model's Bk.

def trust_region(subproblem_solver, x0, radius0):
    # The algorithm is modified from Algorithm 4.1 in the book.
    xcoords = [x0[0]]
    ycoords = [x0[1]]
    iter    = 0
    x = x0 
    radius = radius0
    err = norm(grad_objective_func(x))
    while (err > 1e-9):
        # main iterations for trust region.
        p = subproblem_solver(x, radius)
        # then test whether we should accept this point, calculate the ratio
        r = rho(x, p)
        if r < 0.25:
            # reject this because it is too small, x is not updated, only radius is updated
            radius = 0.5 * radius 
        elif r > 0.75:
            # accept this 
            radius = min(2 * radius , radius0) # cannot exceed maximum region size
            x = x + p
        else:
            # we also accept this, but region size is not changed
            x = x + p
            
        xcoords.append(x[0])
        ycoords.append(x[1])
        iter = iter + 1
            
        # update err
        err = norm(grad_objective_func(x))
            
    return x, iter, xcoords, ycoords

In [28]:
def cauchy_point(x, radius):
    grad = grad_objective_func(x)
    hess = hessian_func(x)
    if multi_dot([grad.T, hess, grad]) <= 0:
        return -radius / norm(grad) * grad 
    else:
        k = norm(grad)**3 / (radius * multi_dot([grad.T, hess, grad]))
        return min(1, k) * (-radius / norm(grad) * grad)

### Trust region method with dogleg method

In [29]:
def dogleg(x, radius):
    grad = grad_objective_func(x)
    hess = hessian_func(x)   
    pn = -solve(hess, grad) 
    if norm(pn)<= radius:
        return pn 
    else:
        ps = -(norm(grad)**2)/(multi_dot([grad.T, hess, grad])) * grad
        if norm(ps) <= radius:
            A = norm(ps - pn)**2 
            B = 2 * np.dot(ps, ps - pn)
            C = norm(ps)**2 - radius**2 
            t = (-B + np.sqrt(B**2 - 4 * A * C))/(2 * A)
            return (1-t)*ps + t*pn
        else:
            return -radius / norm(grad) * grad

### BFGS method with exact line search

In [30]:
tol = 1e-9

def exact_line_search_quasi_newton(update_method, x0, H0):
    k = 0
    xcoords = [x0[0]]
    ycoords = [x0[1]]
    x_k = x0 
    H_k = H0 
    g_k = grad_objective_func(x_k)
    while norm(g_k) > tol:
        p_k = -np.matmul(H_k, g_k)                # search direction
   
        def subproblem1D(alpha):                  # for exact line search
            return objective_func(x_k + alpha * p_k)
        
        res = minimize_scalar(subproblem1D)
        alpha_k = res.x 

        s_k     = alpha_k * p_k                   # s_k = x_{k+1} - x_k 
        g_k1    = grad_objective_func(x_k + s_k)  # gk1 is g_{k+1}
        y_k     = g_k1 - g_k                      # y_k = g_{k+1} - g_(k)
        
        k = k + 1                                 # increment
        H_k = update_method(H_k, s_k, y_k).A      
        x_k = x_k + s_k 
        g_k = g_k1
        
        xcoords.append(x_k[0])
        ycoords.append(x_k[1])
        
    return x_k, k, norm(g_k), xcoords, ycoords

In [31]:
def BFGS(H, s_k, y_k):
    I = np.eye(H.shape[0])
    y_vec = y_k[:, None] # want to vectorize for matrix multiplication
    s_vec = s_k[:, None]
    rho = 1/ np.dot(y_k,s_k) # this is a scalar so multiply with array form
    l = I - rho*np.dot(s_vec, y_vec.T)
    r = I - rho*np.dot(y_vec, s_vec.T)
    H = multi_dot([l,H,r]) + rho*np.dot(s_vec,s_vec.T)
    return np.matrix(H)

###  SR1 method with exact line search

In [32]:
def SR1(H, s_k, y_k):
    z = s_k - np.dot(H, y_k) 
    # the numerator is a matrix!
    numer = np.matrix(z).T * np.matrix(z)
    denom = np.dot(z, y_k)
    # when the denominator is too small, we skip the iteration.
    if np.abs(denom) < 10**(-8) * norm(z) * norm(y_k):
        return H
    else:
        return H + numer/denom 

### BFGS and SR1 method with backtracking line search

In [33]:
def backtracking_line_search_quasi_newton(update_method, x0, H0):
    k = 0
    xcoords = [x0[0]]
    ycoords = [x0[1]]
    x_k = x0 
    H_k = H0 
    g_k = grad_objective_func(x_k)
    while norm(g_k) > 1e-9:
        p_k = -np.matmul(H_k, g_k)                # search direction
        alpha = backtracking(objective_func, grad_objective_func, x_k, p_k, 1, 1e-4, 0.9)      
        s_k     = alpha * p_k                     # s_k = x_{k+1} - x_k 
        g_k1    = grad_objective_func(x_k + s_k)  # gk1 is g_{k+1}
        y_k     = g_k1 - g_k                      # y_k = g_{k+1} - g_(k)
        
        k = k + 1                                 # increment
        H_k = update_method(H_k, s_k, y_k).A      
        x_k = x_k + s_k 
        g_k = g_k1
        
        xcoords.append(x_k[0])
        ycoords.append(x_k[1])
        
    return x_k, k, norm(g_k), xcoords, ycoords

### Problem 1: Quadratic function, starting at (5,5)

In [34]:
# Setting up objective and its gradient function
def objective_func(x):
    return (x[0]+2*x[1]-7)**2+(2*x[0]+x[1]-5)**2

def grad_objective_func(x):
    return np.array([10*x[0]+8*x[1]-34, 8*x[0]+10*x[1]-38])

def hessian_func(x):
    return np.matrix([[10, 8], [8, 10]])

In [35]:
x0 = np.array([5, 5])
H0 = np.eye(2)
i, x = exact_steepest_descent(x0)
print("Iteration for exact steepest descent:", i)
i, x = inexact_steepest_descent(x0)
print("Iteration for fixed step length steepest descent:", i)
i, x = newton_method(x0)
print("Iteration for newton's method:", i)
xk, k, path = backtracking_steepest_decent_method(objective_func, grad_objective_func, x0, tol, maxIter)
print("Iteration for backtracking steepest descent:", k)
xk, k, path = backtracking_newton_method(objective_func, grad_objective_func, hessian_func, x0, tol, maxIter)
print("Iteration for backtracking newton:", k)
xk, k, path = heavyBall(objective_func, grad_objective_func, x0, tol, maxIter)
print("Iteration for heavyball:", k)
x, iter, xcoords, ycoords  = trust_region(cauchy_point, x0, 2)
print("Iteration for trust region w/ cauchy point:", iter)
x, iter, xcoords, ycoords  = trust_region(dogleg, x0, 2)
print("Iteration for trust region w/ dogleg:", iter)
x, iter, err, xcoords, ycoords = exact_line_search_quasi_newton(BFGS, x0, H0)
print("Iteration for BFGS w/ exact line search:", iter)
x, iter, err, xcoords, ycoords = backtracking_line_search_quasi_newton(BFGS, x0, H0)
print("Iteration for BFGS w/ backtracking:", iter)
x, iter, err, xcoords, ycoords = exact_line_search_quasi_newton(SR1, x0, H0)
print("Iteration for SR1 w/ exact line search:", iter)
x, iter, err, xcoords, ycoords = backtracking_line_search_quasi_newton(SR1, x0, H0)
print("Iteration for SR1 w/ backtracking:", iter)


Iteration for exact steepest descent: 12
Iteration for fixed step length steepest descent: 10872
Iteration for newton's method: 1
Iteration for backtracking steepest descent: 113
Iteration for backtracking newton: 1
Iteration for heavyball: 826
Iteration for trust region w/ cauchy point: 77
Iteration for trust region w/ dogleg: 3
Iteration for BFGS w/ exact line search: 2
Iteration for BFGS w/ backtracking: 6
Iteration for SR1 w/ exact line search: 2
Iteration for SR1 w/ backtracking: 3


### Problem 2: Rosenbrock function, starting at (−1.2,1)

In [36]:
def objective_func(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2

def grad_objective_func(x):
    return np.array([-400 * x[0] * (x[1] - x[0] ** 2) - 2 * (1 - x[0]), 200 * (x[1] - x[0]**2)])

def hessian_func(x):
    return np.matrix([[-400*(x[1] - 3*x[0]**2) + 2, -400 * x[0]], [-400 * x[0], 200]])

In [37]:
x0 = np.array([-1.2, 1.])
H0 = np.eye(2)

i, x = exact_steepest_descent(x0)
print("Iteration for exact steepest descent:", i)
i, x = inexact_steepest_descent(x0)
print("Iteration for fixed step length steepest descent:", i)
i, x = newton_method(x0)
print("Iteration for newton's method:", i)
xk, k, path = backtracking_steepest_decent_method(objective_func, grad_objective_func, x0, tol, maxIter)
print("Iteration for backtracking steepest descent:", k)
xk, k, path = backtracking_newton_method(objective_func, grad_objective_func, hessian_func, x0, tol, maxIter)
print("Iteration for backtracking newton:", k)
xk, k, path = heavyBall(objective_func, grad_objective_func, x0, tol, maxIter)
print("Iteration for heavyball:", k)
x, iter, xcoords, ycoords  = trust_region(cauchy_point, x0, 2)
print("Iteration for trust region w/ cauchy point:", iter)
x, iter, xcoords, ycoords  = trust_region(dogleg, x0, 2)
print("Iteration for trust region w/ dogleg:", iter)
x, iter, err, xcoords, ycoords = exact_line_search_quasi_newton(BFGS, x0, H0)
print("Iteration for BFGS w/ exact line search:", iter)
x, iter, err, xcoords, ycoords = backtracking_line_search_quasi_newton(BFGS, x0, H0)
print("Iteration for BFGS w/ backtracking:", iter)
x, iter, err, xcoords, ycoords = exact_line_search_quasi_newton(SR1, x0, H0)
print("Iteration for SR1 w/ exact line search:", iter)
x, iter, err, xcoords, ycoords = backtracking_line_search_quasi_newton(SR1, x0, H0)
print("Iteration for SR1 w/ backtracking:", iter)



Iteration for exact steepest descent: 24500
Iteration for fixed step length steepest descent: 49371
Iteration for newton's method: 7
Iteration for backtracking steepest descent: 946
Iteration for backtracking newton: 20
Iteration for heavyball: 4682
Iteration for trust region w/ cauchy point: 23176
Iteration for trust region w/ dogleg: 38
Iteration for BFGS w/ exact line search: 22
Iteration for BFGS w/ backtracking: 21
Iteration for SR1 w/ exact line search: 22


AssertionError: 

### Problem 3: Powell’s quartic function, starting at (3, −1, 0, 1)

In [73]:
def objective_func(x):
    return (x[0] + 10*x[1])**2 + 5*(x[2] - x[3])**2 + (x[1] - 2*x[2])**4 + 10*(x[0] - x[3])**4

def grad_objective_func(x):
    return np.array([2*(x[0] + 10*x[1]) + 40*(x[0] - x[3])**3, 20*(x[0] + 10*x[1]) + 4*(x[1] - 2*x[2])**3, 
                     10*(x[2] - x[3]) - 8*(x[1] - 2*x[2])**3, -10*(x[2] - x[3]) - 40*(x[0] - x[3])**3])
def hessian_func(x):
    return np.matrix([[2 + 120*(x[0] - x[3])**2, 20, 0, -120*(x[0] - x[3])**2], 
                      [20, 200 + 12*(x[1] - 2*x[2])**2, -24*(x[1] - 2*x[2])**2, 0],
                      [0, -24*(x[1] - 2*x[2])**2, 10 + 48*(x[1] - 2*x[2])**2, -10],
                      [-120*(x[0] - x[3])**2, 0, -10, 10 + 120*(x[0] - x[3])**2]])


In [75]:
x0 = np.array([3, -1, 0, 1])
H0 = np.eye(4)



x, iter, err, xcoords, ycoords = exact_line_search_quasi_newton(BFGS, x0, H0)
print("Iteration for BFGS w/ exact line search:", iter)
x, iter, err, xcoords, ycoords = backtracking_line_search_quasi_newton(BFGS, x0, H0)
print("Iteration for BFGS w/ backtracking:", iter)
x, iter, err, xcoords, ycoords = exact_line_search_quasi_newton(SR1, x0, H0)
print("Iteration for SR1 w/ exact line search:", iter)
x, iter, err, xcoords, ycoords = backtracking_line_search_quasi_newton(SR1, x0, H0)
print("Iteration for SR1 w/ backtracking:", iter)



Iteration for BFGS w/ exact line search: 32
Iteration for BFGS w/ backtracking: 58
Iteration for SR1 w/ exact line search: 32


AssertionError: 